# Hands-on tutorial for AV-HuBERT

In this notebook, we show-case how to use pre-trained models for:
* lip reading
* feature extraction

## Preliminaries
This section installs necessary python packages for the other sections. Run it first.

In [1]:
%cd /content/
!git clone https://github.com/facebookresearch/av_hubert.git

%cd av_hubert
!git submodule init
!git submodule update
!pip install scipy
!pip install sentencepiece
!pip install python_speech_features
!pip install scikit-video

%cd fairseq
!pip install ./

/content
Cloning into 'av_hubert'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 115 (delta 0), reused 2 (delta 0), pack-reused 111
Receiving objects: 100% (115/115), 4.62 MiB | 8.18 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/av_hubert
Submodule 'fairseq' (https://github.com/pytorch/fairseq) registered for path 'fairseq'
Cloning into '/content/av_hubert/fairseq'...
Submodule path 'fairseq': checked out 'afc77bdf4bb51453ce76f1572ef2ee6ddcda8eeb'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel f

In [2]:
import os
import subprocess
import urllib.request

# Clone the Github repository
subprocess.call(['git', 'clone', 'https://github.com/Markfryazino/wav2lip-hq.git'])
os.chdir('wav2lip-hq')

# Install dependencies
subprocess.call(['pip3', 'install', 'gdown'])
subprocess.call(['pip3', 'install', '-r', 'requirements.txt'])

# Download the face detection model
url = "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth"
filename = "face_detection/detection/sfd/s3fd.pth"
urllib.request.urlretrieve(url, filename)


('face_detection/detection/sfd/s3fd.pth',
 <http.client.HTTPMessage at 0x157589ee1a0>)

## Download an example video
This section illustrates how to prepare a video clip.

1. Install necessary tools for preprocessing

In [2]:
!mkdir -p /content/data/misc/
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 -O /content/data/misc/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d /content/data/misc/shape_predictor_68_face_landmarks.dat.bz2
!wget --content-disposition https://github.com/mpc001/Lipreading_using_Temporal_Convolutional_Networks/raw/master/preprocessing/20words_mean_face.npy -O /content/data/misc/20words_mean_face.npy

--2023-03-30 00:54:44--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘/content/data/misc/shape_predictor_68_face_landmarks.dat.bz2’

/content/data/misc/ 100%[===================>]  61.07M  18.7MB/s    in 3.6s    

2023-03-30 00:54:48 (17.0 MB/s) - ‘/content/data/misc/shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]

--2023-03-30 00:55:07--  https://github.com/mpc001/Lipreading_using_Temporal_Convolutional_Networks/raw/master/preprocessing/20words_mean_face.npy
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mpc001/Lipreading_using_Temporal_Convolutional_Networks/master/preprocessing/20words_me

2. Download a video from online

In [3]:
!wget --content-disposition "https://drive.google.com/uc?export=download&id=1Z03hSH13sIvO2ulPxbDQ5MKDm4So8Paw" -O /content/data/clip.mp4

--2023-03-30 00:55:08--  https://drive.google.com/uc?export=download&id=1Z03hSH13sIvO2ulPxbDQ5MKDm4So8Paw
Resolving drive.google.com (drive.google.com)... 142.250.125.100, 142.250.125.101, 142.250.125.113, ...
Connecting to drive.google.com (drive.google.com)|142.250.125.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-ac-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/a7flh2kmeeaeqlr17dmqr9cdalrpk14g/1680137700000/15018073120057225538/*/1Z03hSH13sIvO2ulPxbDQ5MKDm4So8Paw?e=download&uuid=bfd85612-1bc7-4e3e-a826-a4c56b9127bf [following]
--2023-03-30 00:55:09--  https://doc-10-ac-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/a7flh2kmeeaeqlr17dmqr9cdalrpk14g/1680137700000/15018073120057225538/*/1Z03hSH13sIvO2ulPxbDQ5MKDm4So8Paw?e=download&uuid=bfd85612-1bc7-4e3e-a826-a4c56b9127bf
Resolving doc-10-ac-docs.googleusercontent.com (doc-10-ac-docs.googleusercontent.com)... 142.250.125.132, 

3. Extract mouth ROI.

In [4]:
%cd /content/av_hubert/avhubert/

import dlib, cv2, os
import numpy as np
import skvideo
import skvideo.io
from tqdm import tqdm
from preparation.align_mouth import landmarks_interpolate, crop_patch, write_video_ffmpeg
from IPython.display import HTML
from base64 import b64encode

def play_video(video_path, width=200):
  mp4 = open(video_path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML(f"""
  <video width={width} controls>
        <source src="{data_url}" type="video/mp4">
  </video>
  """)

def detect_landmark(image, detector, predictor):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    rects = detector(gray, 1)
    coords = None
    for (_, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        coords = np.zeros((68, 2), dtype=np.int32)
        for i in range(0, 68):
            coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords

def preprocess_video(input_video_path, output_video_path, face_predictor_path, mean_face_path):
  detector = dlib.get_frontal_face_detector()
  predictor = dlib.shape_predictor(face_predictor_path)
  STD_SIZE = (256, 256)
  mean_face_landmarks = np.load(mean_face_path)
  stablePntsIDs = [33, 36, 39, 42, 45]
  videogen = skvideo.io.vread(input_video_path)
  frames = np.array([frame for frame in videogen])
  landmarks = []
  for frame in tqdm(frames):
      landmark = detect_landmark(frame, detector, predictor)
      landmarks.append(landmark)
  preprocessed_landmarks = landmarks_interpolate(landmarks)
  rois = crop_patch(input_video_path, preprocessed_landmarks, mean_face_landmarks, stablePntsIDs, STD_SIZE, 
                        window_margin=12, start_idx=48, stop_idx=68, crop_height=96, crop_width=96)
  write_video_ffmpeg(rois, output_video_path, "/usr/bin/ffmpeg")
  return

/content/av_hubert/avhubert


In [5]:
play_video('/content/data/clip.mp4', width=300)

In [6]:
face_predictor_path = "/content/data/misc/shape_predictor_68_face_landmarks.dat"
mean_face_path = "/content/data/misc/20words_mean_face.npy"
origin_clip_path = "/content/data/clip.mp4"
mouth_roi_path = "/content/data/roi.mp4"
preprocess_video(origin_clip_path, mouth_roi_path, face_predictor_path, mean_face_path)
play_video(mouth_roi_path)

100%|██████████| 74/74 [00:24<00:00,  3.00it/s]


## Import a pre-trained model
This section illustrates how to load a pre-trained model and use it for inference.

1. Download a model checkpoint

In [7]:
!pwd
%mkdir -p /content/data/
!wget https://dl.fbaipublicfiles.com/avhubert/model/lrs3_vox/vsr/base_vox_433h.pt -O /content/data/finetune-model.pt

/content/av_hubert/avhubert
--2023-03-30 00:55:45--  https://dl.fbaipublicfiles.com/avhubert/model/lrs3_vox/vsr/base_vox_433h.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 52.84.125.65, 52.84.125.87, 52.84.125.2, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|52.84.125.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1928063847 (1.8G) [binary/octet-stream]
Saving to: ‘/content/data/finetune-model.pt’

/content/data/finet 100%[===================>]   1.79G  36.4MB/s    in 51s     

2023-03-30 00:56:37 (36.0 MB/s) - ‘/content/data/finetune-model.pt’ saved [1928063847/1928063847]



2. Inference with the model

In [27]:
%cd /content/av_hubert/avhubert
import cv2
import tempfile
from argparse import Namespace
import fairseq
from fairseq import checkpoint_utils, options, tasks, utils
from fairseq.dataclass.configs import GenerationConfig
from IPython.display import HTML

def predict(video_path, ckpt_path, user_dir):
  num_frames = int(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FRAME_COUNT))
  data_dir = tempfile.mkdtemp()
  tsv_cont = ["/\n", f"test-0\t{video_path}\t{None}\t{num_frames}\t{int(16_000*num_frames/25)}\n"]
  label_cont = ["DUMMY\n"]
  with open(f"{data_dir}/test.tsv", "w") as fo:
    fo.write("".join(tsv_cont))
  with open(f"{data_dir}/test.wrd", "w") as fo:
    fo.write("".join(label_cont))
  utils.import_user_module(Namespace(user_dir=user_dir))
  modalities = ["video"]
  gen_subset = "test"
  gen_cfg = GenerationConfig(beam=20)
  models, saved_cfg, task = checkpoint_utils.load_model_ensemble_and_task([ckpt_path])
  models = [model.eval() for model in models]
  saved_cfg.task.modalities = modalities
  saved_cfg.task.data = data_dir
  saved_cfg.task.label_dir = data_dir
  task = tasks.setup_task(saved_cfg.task)
  task.load_dataset(gen_subset, task_cfg=saved_cfg.task)
  generator = task.build_generator(models, gen_cfg)

  def decode_fn(x):
      dictionary = task.target_dictionary
      symbols_ignore = generator.symbols_to_strip_from_output
      symbols_ignore.add(dictionary.pad())
      return task.datasets[gen_subset].label_processors[0].decode(x, symbols_ignore)

  itr = task.get_batch_iterator(dataset=task.dataset(gen_subset)).next_epoch_itr(shuffle=False)
  sample = next(itr)
  # sample = utils.move_to_cuda(sample)
  hypos = task.inference_step(generator, models, sample)
  ref = decode_fn(sample['target'][0].int().cpu())
  hypo = hypos[0][0]['tokens'].int().cpu()
  hypo = decode_fn(hypo)
  return hypo

mouth_roi_path, ckpt_path = "/content/data/roi.mp4", "/content/data/finetune-model.pt"
user_dir = "/content/av_hubert/avhubert"
hypo = predict(mouth_roi_path, ckpt_path, user_dir)
HTML(f"""
  <h3>
    Prediction - {hypo}
  </h3>
  """)

/content/av_hubert/avhubert


3. Extract visual feature with the model

In [ ]:
%cd /content/av_hubert/avhubert
import cv2
import tempfile
import torch
import utils as avhubert_utils
from argparse import Namespace
import fairseq
from fairseq import checkpoint_utils, options, tasks, utils
from IPython.display import HTML

def extract_visual_feature(video_path, ckpt_path, user_dir, is_finetune_ckpt=False):
  utils.import_user_module(Namespace(user_dir=user_dir))
  models, saved_cfg, task = checkpoint_utils.load_model_ensemble_and_task([ckpt_path])
  transform = avhubert_utils.Compose([
      avhubert_utils.Normalize(0.0, 255.0),
      avhubert_utils.CenterCrop((task.cfg.image_crop_size, task.cfg.image_crop_size)),
      avhubert_utils.Normalize(task.cfg.image_mean, task.cfg.image_std)])
  frames = avhubert_utils.load_video(video_path)
  print(f"Load video {video_path}: shape {frames.shape}")
  frames = transform(frames)
  print(f"Center crop video to: {frames.shape}")
  frames = torch.FloatTensor(frames).unsqueeze(dim=0).unsqueeze(dim=0).cuda()
  model = models[0]
  if hasattr(models[0], 'decoder'):
    print(f"Checkpoint: fine-tuned")
    model = models[0].encoder.w2v_model
  else:
    print(f"Checkpoint: pre-trained w/o fine-tuning")
  model.cuda()
  model.eval()
  with torch.no_grad():
    # Specify output_layer if you want to extract feature of an intermediate layer
    feature, _ = model.extract_finetune(source={'video': frames, 'audio': None}, padding_mask=None, output_layer=None)
    feature = feature.squeeze(dim=0)
  print(f"Video feature shape: {feature.shape}")
  return feature

mouth_roi_path, ckpt_path = "/content/data/roi.mp4", "/content/data/finetune-model.pt"
user_dir = "/content/av_hubert/avhubert"
feature = extract_visual_feature(mouth_roi_path, ckpt_path, user_dir)

/content/av_hubert/avhubert
Load video /content/data/roi.mp4: shape (240, 96, 96)
Center crop video to: (240, 88, 88)
Checkpoint: fine-tuned
Video feature shape: torch.Size([240, 768])
